#Imported Libraries

In [ ]:
!pip install SpeechRecognition
!pip install google-cloud-speech pandas
!pip install pydub
!apt install ffmpeg
!pip install FER

  Attempting uninstall: imageio
    Found existing installation: imageio 2.31.6
    Uninstalling imageio-2.31.6:
      Successfully uninstalled imageio-2.31.6


In [ ]:
import os
import speech_recognition as sr
import spacy
import pandas as pd
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from google.cloud import speech
import pandas as pd
import os
import glob
from pydub import AudioSegment

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Labelled Sentiments

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/work2/updated_transcriptions_with_senti.csv')
df.head(111)

,File,Transcription,Status,Length,WordCount,Senti
0,1.wav,پلیز ایک مشہور برینڈ ہے اور لوگوں کو بہت پسند ...,Transcribed,21.04,42,0
1,2.wav,پلیز میرا پسندیدہ اسنیک ہے اور ہر بائیک مجھے خ...,Transcribed,23.07,34,1
2,3.wav,میرا نام اسان عبداللہ اور اج مجھ سے رزق لیز کا...,Transcribed,28.16,85,2
3,4.wav,السلام علیکم میرا نام اسان عبداللہ ہے اور اج س...,Transcribed,26.74,66,1
4,5.wav,السلام علیکم میرا نام ارشد ہے ماجد ہے اور اج م...,Transcribed,29.31,82,2
...,...,...,...,...,...,...
106,108.wav,السلام علیکم اج موسی پیسوں کا ریویو لیا جارہا ...,Transcribed,30.251,90,0
107,109.wav,اپ مجھ سے پیپسی کا ریویو لیا جا رہا ہے پیپسی ب...,Transcribed,37.61,113,2
108,110.wav,مولانا مثان عبداللہ ہے اور اج مجھ سے ایک پیچی ...,Transcribed,27.07,73,1
109,111.wav,السلام علیکم اج مجھ سے ریلیز کا ریویو لیے جارہ...,Transcribed,22.72,70,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   File           111 non-null    object
 1   Transcription  111 non-null    object
 2   Status         111 non-null    object
 3   Length         111 non-null    object
 4   WordCount      111 non-null    object
 5   Senti          111 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 5.3+ KB


#Saved Negative Positions and Positive Positions of Words

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_final.csv')
df.head(100)

,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions,Senti
0,21.04,1.wav,پلیز ایک مشہور برینڈ ہے اور لوگوں کو بہت پسند ...,1,7,[11],"[2, 5, 9, 9, 10, 18, 20]",0
1,23.07,2.wav,پلیز میرا پسندیدہ اسنیک ہے اور ہر بائیک مجھے خ...,2,3,"[7, 21]","[2, 16, 19]",1
2,28.16,3.wav,میرا نام اسان عبداللہ اور اج مجھ سے رزق لیز کا...,4,3,"[7, 19, 23, 25]","[6, 15, 22]",2
3,26.74,4.wav,السلام علیکم میرا نام اسان عبداللہ ہے اور اج ا...,0,3,[],"[8, 15, 17]",1
4,29.31,5.wav,السلام علیکم میرا نام ارشد ہے ماجد ہے اور اج م...,3,3,"[6, 18, 28]","[16, 21, 24]",2
...,...,...,...,...,...,...,...,...
95,27.62,96.wav,ہاں جی تو پانی پی پی کر اپ اپنی زندگی سے بیزار...,3,1,"[7, 13, 14]",[6],1
96,32.96,97.wav,السلام علیکم میرا نام بلال لک پھیرا ہے اور اج ...,0,3,[],"[24, 30, 32]",1
97,35.81,98.wav,السلام علیکم میرا نام بلال لکیر ہے اور اج کا ع...,8,3,"[7, 8, 13, 14, 18, 22, 27, 33]","[14, 20, 34]",1
98,31.28,99.wav,جیسا کہ میں نے اپ کو پیپسی کے بارے میں بتایا ا...,11,8,"[6, 12, 19, 19, 20, 21, 24, 25, 25, 28, 28]","[4, 12, 15, 16, 21, 24, 30, 30]",1


#Extracted Frames and saved Path of files

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/work2/Final/extracted_frames_info.csv')

# Display the DataFrame
df.head(10)

,Video Name,Frame Path,Time in Seconds
0,46,/content/drive/MyDrive/work2/Final/Frames/46/4...,3
1,46,/content/drive/MyDrive/work2/Final/Frames/46/4...,13
2,46,/content/drive/MyDrive/work2/Final/Frames/46/4...,15
3,35,/content/drive/MyDrive/work2/Final/Frames/35/3...,1
4,35,/content/drive/MyDrive/work2/Final/Frames/35/3...,12
5,35,/content/drive/MyDrive/work2/Final/Frames/35/3...,14
6,35,/content/drive/MyDrive/work2/Final/Frames/35/3...,21
7,35,/content/drive/MyDrive/work2/Final/Frames/35/3...,27
8,35,/content/drive/MyDrive/work2/Final/Frames/35/3...,28
9,25,/content/drive/MyDrive/work2/Final/Frames/25/2...,1


#Frames Analysis using FER

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_with_compiled_sentiments.csv')
df.head()

,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions,Senti,Negative_Frame_Sentiments,Positive_Frame_Sentiments
0,21.04,1.wav,پلیز ایک مشہور برینڈ ہے اور لوگوں کو بہت پسند ...,1,7,[11],"[2, 5, 9, 9, 10, 18, 20]",0,[9],"[9, 0, 0, 1, 0, 0]"
1,23.07,2.wav,پلیز میرا پسندیدہ اسنیک ہے اور ہر بائیک مجھے خ...,2,3,"[7, 21]","[2, 16, 19]",1,"[0, 9]","[0, 9, 0]"
2,28.16,3.wav,میرا نام اسان عبداللہ اور اج مجھ سے رزق لیز کا...,4,3,"[7, 19, 23, 25]","[6, 15, 22]",2,"[0, 0, 0, 9]","[0, 1, 0]"
3,26.74,4.wav,السلام علیکم میرا نام اسان عبداللہ ہے اور اج ا...,0,3,[],"[8, 15, 17]",1,[],"[0, 1, 0]"
4,29.31,5.wav,السلام علیکم میرا نام ارشد ہے ماجد ہے اور اج م...,3,3,"[6, 18, 28]","[16, 21, 24]",2,"[0, 0, 0]","[0, 0, 0]"


#Final Analysis with General Percentage

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_with_video_analysis.csv')
df.head(100)

,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions,Senti,Total Frame Count,Total Positive Frames,Total Negative Frames,Total Neutral Frames,General Percentage
0,21.04,1.wav,پلیز ایک مشہور برینڈ ہے اور لوگوں کو بہت پسند ...,1,7,[11],"[2, 5, 9, 9, 10, 18, 20]",0,20,0,1,3,"(0, 15.0)"
1,23.07,2.wav,پلیز میرا پسندیدہ اسنیک ہے اور ہر بائیک مجھے خ...,2,3,"[7, 21]","[2, 16, 19]",1,23,10,0,12,"(0, 52.17391304347826)"
2,28.16,3.wav,میرا نام اسان عبداللہ اور اج مجھ سے رزق لیز کا...,4,3,"[7, 19, 23, 25]","[6, 15, 22]",2,28,0,1,27,"(0, 96.42857142857143)"
3,26.74,4.wav,السلام علیکم میرا نام اسان عبداللہ ہے اور اج ا...,0,3,[],"[8, 15, 17]",1,26,0,0,26,"(0, 100.0)"
4,29.31,5.wav,السلام علیکم میرا نام ارشد ہے ماجد ہے اور اج م...,3,3,"[6, 18, 28]","[16, 21, 24]",2,29,2,0,19,"(0, 65.51724137931035)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,27.62,96.wav,ہاں جی تو پانی پی پی کر اپ اپنی زندگی سے بیزار...,3,1,"[7, 13, 14]",[6],1,27,2,0,22,"(0, 81.48148148148148)"
96,32.96,97.wav,السلام علیکم میرا نام بلال لک پھیرا ہے اور اج ...,0,3,[],"[24, 30, 32]",1,32,4,0,26,"(0, 81.25)"
97,35.81,98.wav,السلام علیکم میرا نام بلال لکیر ہے اور اج کا ع...,8,3,"[7, 8, 13, 14, 18, 22, 27, 33]","[14, 20, 34]",1,35,3,0,32,"(0, 91.42857142857143)"
98,31.28,99.wav,جیسا کہ میں نے اپ کو پیپسی کے بارے میں بتایا ا...,11,8,"[6, 12, 19, 19, 20, 21, 24, 25, 25, 28, 28]","[4, 12, 15, 16, 21, 24, 30, 30]",1,31,8,5,18,"(0, 58.06451612903226)"


#Final Analysis of Frames with frames positions

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/Final/Final_results.csv')
df.head(100)

NameError: name 'pd' is not defined

#26 Video Analysis

**Audio Transcription**

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import os
import pandas as pd

def get_audio_length(file_path):
    """Return the length of the audio file in seconds."""
    audio = AudioSegment.from_file(file_path)
    return len(audio) / 1000.0  # Convert milliseconds to seconds

def transcribe_audio_speech_recognition(audio_file_path):
    """Transcribe the given local audio file using the speech_recognition library."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        try:
            transcript = recognizer.recognize_google(audio_data, language="ur-PK")
            return transcript
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

def read_words_from_file(file_path):
    """Read words from a file and return them as a list."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return [line.strip().lower() for line in file.readlines()]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

audio_file_path = '/content/drive/MyDrive/work2/mono/26.wav'
length_in_seconds = get_audio_length(audio_file_path)
print(f"The length of the audio file is {length_in_seconds} seconds.")

transcript = transcribe_audio_speech_recognition(audio_file_path)
if transcript:
    print("Transcript:", transcript)
    words_in_transcript = transcript.split()
    transcript_length = len(words_in_transcript)
    print(f"Transcript Length (in words): {transcript_length}")

    negative_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_negative_words.txt')
    positive_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_positive_words.txt')

    results = []
    negative_count, positive_count = 0, 0
    for index, word in enumerate(words_in_transcript):
        position = (index+1) / transcript_length * length_in_seconds
        if word.lower() in negative_words:
            negative_count += 1
            results.append({'Word': word, 'Type': 'Negative', 'Position': index+1, 'Time': round(position)})
        elif word.lower() in positive_words:
            positive_count += 1
            results.append({'Word': word, 'Type': 'Positive', 'Position': index+1, 'Time': round(position)})

    # Create DataFrame and display results
    df = pd.DataFrame(results)
    print(df)
    print(f"Total Negative Words: {negative_count}")
    print(f"Total Positive Words: {positive_count}")

else:
    print("No transcript available.")


The length of the audio file is 30.43 seconds.
Transcript: السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چیزوں کے بارے میں بتاؤں گی لیز دنیا بھر میں بہت پسند کیے جاتے ہیں اور لیز کے پیکٹ کے اندر ایک خاص قسم کی ہوا ہوتی ہے جو اپ کے لئے اس کو تازہ اور خستہ رکھتی ہے میرے گھر میں مصالحہ میں بہت پسند کیے جاتے ہیں اور اپ کو کون سا فلیور اچھا لگتا ہے پلیز مجھے بتائیے اور اللہ حافظ
Transcript Length (in words): 74
   Word      Type  Position  Time
0  اچھی  Positive        10     4
1   بھر  Positive        19     8
2  پسند  Positive        22     9
3  پیکٹ  Negative        29    12
4   قسم  Positive        34    14
5   ہوا  Positive        36    15
6  خستہ  Positive        47    19
7  پسند  Positive        56    23
8  اچھا  Positive        66    27
Total Negative Words: 1
Total Positive Words: 8


In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import os
import pandas as pd

def get_audio_length(file_path):
    """Return the length of the audio file in seconds."""
    audio = AudioSegment.from_file(file_path)
    return len(audio) / 1000.0  # Convert milliseconds to seconds

def transcribe_audio_speech_recognition(audio_file_path):
    """Transcribe the given local audio file using the speech_recognition library."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        try:
            transcript = recognizer.recognize_google(audio_data, language="ur-PK")
            return transcript
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

def read_words_from_file(file_path):
    """Read words from a file and return them as a list."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return [line.strip().lower() for line in file.readlines()]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

audio_file_path = '/content/drive/MyDrive/work2/mono/26.wav'
length_in_seconds = get_audio_length(audio_file_path)
print(f"The length of the audio file is {length_in_seconds} seconds.")

transcript = transcribe_audio_speech_recognition(audio_file_path)
if transcript:
    print("Transcript:", transcript)
    words_in_transcript = transcript.split()
    transcript_length = len(words_in_transcript)
    print(f"Transcript Length (in words): {transcript_length}")

    negative_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_negative_words.txt')
    positive_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_positive_words.txt')

    results37 = []
    negative_count, positive_count = 0, 0
    negative_positions, positive_positions = [], []  # Lists to store positions

    for index, word in enumerate(words_in_transcript):
        position = (index+1) / transcript_length * length_in_seconds
        if word.lower() in negative_words:
            negative_count += 1
            negative_positions.append(round(position))  # Append to negative_positions list
            # results.append({'Word': word, 'Type': 'Negative', 'Position': index+1, 'Time': round(position)})
        elif word.lower() in positive_words:
            positive_count += 1
            positive_positions.append(round(position))  # Append to positive_positions list
            # results.append({'Word': word, 'Type': 'Positive', 'Position': index+1, 'Time': round(position)})

    print(f"Total Negative Words: {negative_count}")
    print(f"Total Positive Words: {positive_count}")
    print("Negative Word Positions:", negative_positions)
    print("Positive Word Positions:", positive_positions)

    results37.append({'Length': length_in_seconds, 'audio_name': os.path.basename(audio_file_path),'Transcript' : transcript , 'negative_count': negative_count, 'positive_count': positive_count, 'negative_positions' : negative_positions, 'positive_positions': positive_positions})
    # Create DataFrame
    df = pd.DataFrame(results37)

    # Save results to a CSV file
    csv_file_path = '/content/drive/MyDrive/work2/Final_Video/26_Transcription.csv'
    df.to_csv(csv_file_path, index=False)
    print(f"Results saved to {csv_file_path}")



else:
    print("No transcript available.")


The length of the audio file is 30.43 seconds.
Transcript: السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چیزوں کے بارے میں بتاؤں گی لیز دنیا بھر میں بہت پسند کیے جاتے ہیں اور لیز کے پیکٹ کے اندر ایک خاص قسم کی ہوا ہوتی ہے جو اپ کے لئے اس کو تازہ اور خستہ رکھتی ہے میرے گھر میں مصالحہ میں بہت پسند کیے جاتے ہیں اور اپ کو کون سا فلیور اچھا لگتا ہے پلیز مجھے بتائیے اور اللہ حافظ
Transcript Length (in words): 74
Total Negative Words: 1
Total Positive Words: 8
Negative Word Positions: [12]
Positive Word Positions: [4, 8, 9, 14, 15, 19, 23, 27]
Results saved to /content/drive/MyDrive/work2/Final_Video/26_Transcription.csv


In [ ]:
df.head()

,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions
0,30.43,26.wav,السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...,1,8,[12],"[4, 8, 9, 14, 15, 19, 23, 27]"


**Extract Frames**

In [ ]:
import cv2
import pandas as pd
import os

# Video file path
video_file_path = '/content/drive/MyDrive/work2/videos/26.mp4'
video_name = os.path.basename(video_file_path).split('.')[0]  # Extract name without extension

# Create a folder to store frames if it doesn't exist
output_folder = '/content/drive/MyDrive/work2/Final_Video/New_Frames/26'
os.makedirs(output_folder, exist_ok=True)

# Load positions from the CSV file
csv_file_path = '/content/drive/MyDrive/work2/Final_Video/26_Transcription.csv'
df = pd.read_csv(csv_file_path)

# Convert positions to lists of integers
df['negative_positions'] = df['negative_positions'].apply(lambda x: eval(x))
df['positive_positions'] = df['positive_positions'].apply(lambda x: eval(x))

# Concatenate, flatten, and remove duplicates from positions
all_positions = df['negative_positions'] + df['positive_positions']
times_in_seconds = list(set([int(time) for sublist in all_positions for time in sublist]))

# Initialize video capture
cap = cv2.VideoCapture(video_file_path)

frame_info = []  # List to store frame information
fps = cap.get(cv2.CAP_PROP_FPS)

# Extract frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_time = int(cap.get(cv2.CAP_PROP_POS_MSEC) // 1000)  # Current time in seconds

    if current_time in times_in_seconds:
        frame_filename = os.path.join(output_folder, f'{video_name}_frame_{current_time}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_info.append({'Video Name': video_name, 'Frame Path': frame_filename, 'Time in Seconds': current_time})
        times_in_seconds.remove(current_time)  # Remove extracted time

# Release video capture object
cap.release()

# Save frame information to a new CSV file
frame_info_df = pd.DataFrame(frame_info)
frames_csv_path = '/content/drive/MyDrive/work2/Final_Video/26_frames.csv'
frame_info_df.to_csv(frames_csv_path, index=False)

print("Frames extracted and saved in", output_folder)
print(f"Frame information saved to {frames_csv_path}")


Frames extracted and saved in /content/drive/MyDrive/work2/Final_Video/New_Frames/26
Frame information saved to /content/drive/MyDrive/work2/Final_Video/26_frames.csv


In [ ]:
# prompt: /content/drive/MyDrive/work2/Final_Video/26_frames.csv show head

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_frames.csv')
df.head()


,Video Name,Frame Path,Time in Seconds
0,26,/content/drive/MyDrive/work2/Final_Video/New_F...,4
1,26,/content/drive/MyDrive/work2/Final_Video/New_F...,8
2,26,/content/drive/MyDrive/work2/Final_Video/New_F...,9
3,26,/content/drive/MyDrive/work2/Final_Video/New_F...,12
4,26,/content/drive/MyDrive/work2/Final_Video/New_F...,14


In [ ]:


import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_Transcription.csv')
df.head()


,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions
0,30.43,26.wav,السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...,1,8,[12],"[4, 8, 9, 14, 15, 19, 23, 27]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Length              1 non-null      float64
 1   audio_name          1 non-null      object 
 2   Transcript          1 non-null      object 
 3   negative_count      1 non-null      int64  
 4   positive_count      1 non-null      int64  
 5   negative_positions  1 non-null      object 
 6   positive_positions  1 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 184.0+ bytes


In [ ]:
import pandas as pd

# Load the DataFrames
df_transcriptions = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_Transcription.csv')
df_senti = pd.read_csv('/content/drive/MyDrive/work2/updated_transcriptions_with_senti.csv')

# Merge DataFrames on the common key
df_merged = pd.merge(df_transcriptions, df_senti[['File', 'Senti']], left_on='audio_name', right_on='File', how='left')

# Drop the 'File' column from the merged DataFrame
df_merged.drop(columns=['File'], inplace=True)

# Save the updated DataFrame back to a CSV file
updated_csv_path = '/content/drive/MyDrive/work2/Final_Video/26_Transcription_with_Senti.csv'
df_merged.to_csv(updated_csv_path, index=False)

# Display the first few rows of the updated DataFrame
print(df_merged.head())


   Length audio_name                                         Transcript  \
0   30.43     26.wav  السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...   

   negative_count  positive_count negative_positions  \
0               1               8               [12]   

              positive_positions  Senti  
0  [4, 8, 9, 14, 15, 19, 23, 27]      1  
   Length audio_name                                         Transcript  \
0   30.43     26.wav  السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...   

   negative_count  positive_count negative_positions  \
0               1               8               [12]   

              positive_positions  Senti  
0  [4, 8, 9, 14, 15, 19, 23, 27]      1  


In [ ]:
df_merged.head()

,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions,Senti
0,30.43,26.wav,السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...,1,8,[12],"[4, 8, 9, 14, 15, 19, 23, 27]",1


In [ ]:
import pandas as pd
import cv2
from fer import FER

# Function to analyze the most prominent face in a frame and categorize emotions
def analyze_most_prominent_face(frame_path):
    try:
        detector = FER(mtcnn=True)
        image = cv2.imread(frame_path)
        face_detections = detector.detect_emotions(image)

        if face_detections:
            largest_face_area = 0
            positive_sentiment = 0
            negative_sentiment = 0
            neutral_sentiment = 0

            # Find the most prominent face in the frame
            for face in face_detections:
                x, y, w, h = face["box"]
                face_area = w * h
                if face_area > largest_face_area:
                    largest_face_area = face_area
                    emotions = face["emotions"]

                    # Categorize emotions into positive and negative sentiments
                    if any(emotions.get(emotion, 0) > 0.5 for emotion in ['happy', 'surprise']):
                        positive_sentiment = 1
                    if any(emotions.get(emotion, 0) > 0.5 for emotion in ['angry', 'fear', 'disgust']):
                        negative_sentiment = 2
                    if any(emotions.get(emotion, 0) > 0.5 for emotion in ['neutral', 'sad']):
                        neutral_sentiment = 0

            return positive_sentiment, negative_sentiment,neutral_sentiment
        else:
            return 9, 9 ,9  # Neutral for no face
    except Exception as e:
        print(f"Error in analyzing frame {frame_path}: {e}")
        return 'Error', 'Error'

# Load transcription and frame details DataFrames
transcription_df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_Transcription_with_Senti.csv')
frame_details_df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_frames.csv')

# Convert positions in transcription data to lists of integers
transcription_df['negative_positions'] = transcription_df['negative_positions'].apply(lambda x: eval(x))
transcription_df['positive_positions'] = transcription_df['positive_positions'].apply(lambda x: eval(x))

# Initialize sentiment columns in frame details DataFrame
frame_details_df['Positive Sentiment'] = None
frame_details_df['Negative Sentiment'] = None

# Analyze frames for sentiment and assign to corresponding positions
for index, row in frame_details_df.iterrows():
    frame_time = row['Time in Seconds']
    frame_path = row['Frame Path']
    positive_sentiment, negative_sentiment,neutral_sentiment = analyze_most_prominent_face(frame_path)

    if any(frame_time in positions for positions in transcription_df['positive_positions']):
        frame_details_df.at[index, 'Positive Sentiment'] = positive_sentiment
    if any(frame_time in positions for positions in transcription_df['negative_positions']):
        frame_details_df.at[index, 'Negative Sentiment'] = negative_sentiment

# Save updated frame details with FER analysis results
frame_details_with_sentiment_path = '/content/drive/MyDrive/work2/Final_Video/26_frames.csv_details_with_sentiment.csv'
frame_details_df.to_csv(frame_details_with_sentiment_path, index=False)
print(f"Updated frame details with FER analysis saved to {frame_details_with_sentiment_path}")

# Function to compile sentiments into lists for each audio file in transcription data
def compile_sentiments(row, frame_details):
    positive_sentiments = []
    negative_sentiments = []


    for pos in row['negative_positions']:
        sentiment = frame_details.loc[frame_details['Time in Seconds'] == pos, 'Negative Sentiment']
        negative_sentiments.extend(sentiment.values)

    for pos in row['positive_positions']:
        sentiment = frame_details.loc[frame_details['Time in Seconds'] == pos, 'Positive Sentiment']
        positive_sentiments.extend(sentiment.values)



    return pd.Series([negative_sentiments, positive_sentiments])

# Compile sentiments into transcription DataFrame
transcription_df[['Negative_Frame_Sentiments','Positive_Frame_Sentiments']] = transcription_df.apply(
    lambda row: compile_sentiments(row, frame_details_df), axis=1
)

# Save updated transcription results with compiled sentiments
transcription_results_with_compiled_sentiments_path = '/content/drive/MyDrive/work2/Final_Video/26_transcription_results_with_compiled_sentiments.csv'
transcription_df.to_csv(transcription_results_with_compiled_sentiments_path, index=False)
print(f"Transcription results with compiled sentiments saved to {transcription_results_with_compiled_sentiments_path}")


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_frames.csv_details_with_sentiment.csv')
df.head()


,Video Name,Frame Path,Time in Seconds,Positive Sentiment,Negative Sentiment
0,26,/content/drive/MyDrive/work2/Final_Video/New_F...,4,0.0,NaN
1,26,/content/drive/MyDrive/work2/Final_Video/New_F...,8,0.0,NaN
2,26,/content/drive/MyDrive/work2/Final_Video/New_F...,9,1.0,NaN
3,26,/content/drive/MyDrive/work2/Final_Video/New_F...,12,NaN,0.0
4,26,/content/drive/MyDrive/work2/Final_Video/New_F...,14,0.0,NaN


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_transcription_results_with_compiled_sentiments.csv')
df.head()

,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions,Senti,Negative_Frame_Sentiments,Positive_Frame_Sentiments
0,30.43,26.wav,السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...,1,8,[12],"[4, 8, 9, 14, 15, 19, 23, 27]",1,[0],"[0, 0, 1, 0, 0, 0, 0, 0]"


In [ ]:
import pandas as pd

# Load the frame details DataFrame with FER analysis results
frame_details_df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_frames.csv_details_with_sentiment.csv')

# Load the original transcription results DataFrame
transcription_df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_Transcription_with_Senti.csv')

# Function to get positive and negative sentiments for a given audio file
def get_sentiments_for_audio(row, frame_details):
    negative_positions = eval(row['negative_positions'])
    positive_positions = eval(row['positive_positions'])
    all_positions = set(negative_positions + positive_positions)

    positive_sentiments = []
    negative_sentiments = []
    for pos in all_positions:
        sentiment_row = frame_details.loc[frame_details['Time in Seconds'] == pos]
        if not sentiment_row.empty:
            positive_sentiments.append(sentiment_row['Positive Sentiment'].values[0])
            negative_sentiments.append(sentiment_row['Negative Sentiment'].values[0])

    return positive_sentiments, negative_sentiments

# Add sentiments to the transcription data
transcription_df[['Positive_Sentiments_of_Frames', 'Negative_Sentiments_of_Frames']] = transcription_df.apply(
    lambda row: pd.Series(get_sentiments_for_audio(row, frame_details_df)),
    axis=1
)

# Save the updated transcription results with sentiment
transcription_results_with_sentiment_path = '/content/drive/MyDrive/work2/Final_Video/26_Transcription_with_sentiment.csv'
transcription_df.to_csv(transcription_results_with_sentiment_path, index=False)
print(f"Transcription results updated with FER sentiment saved to {transcription_results_with_sentiment_path}")


Transcription results updated with FER sentiment saved to /content/drive/MyDrive/work2/Final_Video/26_Transcription_with_sentiment.csv


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/work2/Final_Video/26_Transcription_with_sentiment.csv')
df


,Length,audio_name,Transcript,negative_count,positive_count,negative_positions,positive_positions,Senti,Positive_Sentiments_of_Frames,Negative_Sentiments_of_Frames
0,30.43,26.wav,السلام علیکم ناظرین اج میں اپ کو لیز کی اچھی چ...,1,8,[12],"[4, 8, 9, 14, 15, 19, 23, 27]",1,"[0.0, 0.0, 1.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, 0.0, nan, nan, nan, nan, nan]"


#Test

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import os
import pandas as pd

def get_audio_length(file_path):
    """Return the length of the audio file in seconds."""
    audio = AudioSegment.from_file(file_path)
    return len(audio) / 1000.0  # Convert milliseconds to seconds

def transcribe_audio_speech_recognition(audio_file_path):
    """Transcribe the given local audio file using the speech_recognition library."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        try:
            transcript = recognizer.recognize_google(audio_data, language="ur-PK")
            return transcript
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

def read_words_from_file(file_path):
    """Read words from a file and return them as a list."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return [line.strip().lower() for line in file.readlines()]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

audio_file_path = '/content/drive/MyDrive/work2/mono/5.wav'
length_in_seconds = get_audio_length(audio_file_path)
print(f"The length of the audio file is {length_in_seconds} seconds.")

transcript = transcribe_audio_speech_recognition(audio_file_path)
if transcript:
    print("Transcript:", transcript)
    words_in_transcript = transcript.split()
    transcript_length = len(words_in_transcript)
    print(f"Transcript Length (in words): {transcript_length}")

    negative_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_negative_words.txt')
    positive_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_positive_words.txt')

    results = []
    negative_count, positive_count = 0, 0
    for index, word in enumerate(words_in_transcript):
        position = (index+1) / transcript_length * length_in_seconds
        if word.lower() in negative_words:
            negative_count += 1
            results.append({'Word': word, 'Type': 'Negative', 'Position': index+1, 'Time': round(position)})
        elif word.lower() in positive_words:
            positive_count += 1
            results.append({'Word': word, 'Type': 'Positive', 'Position': index+1, 'Time': round(position)})

    # Create DataFrame and display results
    df = pd.DataFrame(results)
    print(df)
    print(f"Total Negative Words: {negative_count}")
    print(f"Total Positive Words: {positive_count}")

else:
    print("No transcript available.")


The length of the audio file is 29.31 seconds.
Transcript: السلام علیکم میرا نام ارشد ہے ماجد ہے اور اج میں اپ کو لیس کے کچھ برے باتیں بتاؤں گا لیز فیٹو لے کمپنی کی ایک ایجاد ہے اور یہ جو ہے نا یہ اسرائیل کی ایک کمپنی ہے تو انہوں نے اسرائیل کو سپورٹ کیا ہے ابھی جو بور ہو رہی ہے اسرائیل پہلے اس ٹائم کی کافی عرصے سے چل رہی ہے انہوں نے اسرائیل کو سپورٹ کیا ہے سو اس لیے میں جو ہوں ڈس لائک کروں گا لیز کو
Transcript Length (in words): 83
    Word      Type  Position  Time
0    برے  Negative        17     6
1  سپورٹ  Positive        45    16
2    بور  Negative        50    18
3   کافی  Positive        59    21
4  سپورٹ  Positive        69    24
5     ڈس  Negative        78    28
Total Negative Words: 3
Total Positive Words: 3
